In [9]:
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from dog_class import Mob_Dog
from dog_class import apply_clahe_and_threshold
from dog_class import apply_adaptive_threshold
from dog_class import get_accuracy
from torchvision.transforms.functional import to_pil_image

In [10]:
transform = transforms.Compose([
    transforms.Resize((50, 50)), 
    transforms.Lambda(apply_clahe_and_threshold),  
    transforms.ToTensor()  
])

In [11]:
# 모델 객체 생성, 옵티마이저 생성
model = Mob_Dog()
model = torch.load('model.pth')
model.eval()

Mob_Dog(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1296, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [12]:
import cv2
import numpy as np

isDragging = False                     
x0, y0, w, h = -1,-1,-1,-1              
blue, red = (255,0,0),(0,0,255)        

def onMouse(event,x,y,flags,param):    
    global isDragging, x0, y0, img      # 전역변수 참조
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼쪽 마우스 버튼 다운
        isDragging = True
        x0 = x
        y0 = y
    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 움직임
        if isDragging:                  # 드래그 진행 중
            img_draw = img.copy()
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2) # 드래그 진행 영역 표시
            cv2.imshow('img', img_draw) # 사각형 표시된 그림 화면 출력
    elif event == cv2.EVENT_LBUTTONUP:  # 왼쪽 마우스 버튼 업
        if isDragging:                  # 드래그 중지
            isDragging = False          
            w = x - x0                  # 폭 계산
            h = y - y0                  # 높이 계산
            print("x:%d, y:%d, w:%d, h:%d" % (x0, y0, w, h))
            if w > 0 and h > 0:   
                img_draw = img.copy()   
                cv2.rectangle(img_draw, (x0, y0), (x, y), red, 2) 
                cv2.imshow('img', img_draw) 
                roi = img[y0:y0+h, x0:x0+w] 
                cv2.imshow('cropped', roi)
                cv2.moveWindow('cropped', 0, 0) 
                cv2.imwrite(param, roi)  
                print("croped.")
            else:
                cv2.imshow('img', img)
                print("좌측 상단에서 우측 하단으로 영역을 드래그 하세요.")


try :
    img = cv2.imread("./add_data/mop_dog/0.jpg")
    cv2.imshow('img', img)
    cv2.setMouseCallback('img', onMouse, param="test.jpg")
    cv2.waitKey()
    cv2.destroyAllWindows()
except : 
    pass



x:183, y:28, w:147, h:187
croped.


In [13]:
img = cv2.imread("test.jpg")
img_pil = to_pil_image(img)  

transformed_img = transform(img_pil)  
transformed_img = transformed_img.unsqueeze(0)

with torch.no_grad():
    output = model(transformed_img)
    _, predicted = torch.max(torch.softmax(output, dim=1).data, 1)
    print(f"{torch.softmax(output, dim=1).data[0][predicted].item():.2f} 확률로 {'강쥐!!!' if predicted.item() else '大걸레'}입니다!")


0.88 확률로 강쥐!!!입니다!
